In [31]:
import sys, os

import pandas as pd
import pysal.esda.mapclassify as mapclassify

from mapboxgl.viz import *
from mapboxgl.utils import *
from geopy.geocoders import Nominatim
geolocator = Nominatim()

In [41]:
token = 'token'

In [118]:
# Load data 
dev = pd.read_csv('../Desktop/SF_Development_Pipeline_2017_Q3.csv')
dev.head(1)

,PROJECT_TYPE,BLKLOT,NAMEADDR,Entitled,BESTSTAT,BESTDATE,UNITS,UNITSNET,AFFORDABLE,AFFORDABLENET,...,CONTACT,CONTACTPH,NEIGHBORHOOD,ZONING_SIM,ZONING_GEN,ZONING_DISTRICT,supervisor,PLAN_DISTRICT,PLAN_AREA,Location 1
0,Mixres,4884025,CANDLESTICK POINT,Yes,PL APPROVED,04/10/2014 12:00:00 AM,9108.0,9108.0,3345.0,2833.0,...,Nicole Franklin,NaN,Bayview,P,Public,P,10,10 - South Bayshore,Candlestick Point; Bayview Hunters Point,"(37.7193832397, -122.3849334717)"


In [119]:
dev['lat'] = ' '
dev['lon'] = ' '

In [120]:
for index, row in dev.iterrows():
    loc = row['Location 1']
    lat = float(loc[loc.index('(')+1:loc.index(',')])
    lon = float(loc[loc.index(',')+1:loc.index(')')])
    dev.set_value(index, 'lat', lat)
    dev.set_value(index, 'lon', lon)

In [121]:
dev.head()

,PROJECT_TYPE,BLKLOT,NAMEADDR,Entitled,BESTSTAT,BESTDATE,UNITS,UNITSNET,AFFORDABLE,AFFORDABLENET,...,NEIGHBORHOOD,ZONING_SIM,ZONING_GEN,ZONING_DISTRICT,supervisor,PLAN_DISTRICT,PLAN_AREA,Location 1,lat,lon
0,Mixres,4884025,CANDLESTICK POINT,Yes,PL APPROVED,04/10/2014 12:00:00 AM,9108.0,9108.0,3345.0,2833.0,...,Bayview,P,Public,P,10,10 - South Bayshore,Candlestick Point; Bayview Hunters Point,"(37.7193832397, -122.3849334717)",37.7194,-122.385
1,Mixres,1939001,TREASURE ISLAND DEVELOPMENT AREA,Yes,PL APPROVED,03/15/2011 12:00:00 AM,8619.0,7800.0,2050.0,1800.0,...,Treasure Island/YBI,TI-OS,Public,TI-OS,6,-,Treasure Island,"(37.8242835999, -122.3709487915)",37.8243,-122.371
2,Mixres,7308001,PARKMERCED,Yes,PL APPROVED,02/10/2011 12:00:00 AM,7887.0,4666.0,284.0,284.0,...,Lakeshore,PM-R,Residential,PM-R,7,13 - Ingleside,Parkmerced,"(37.7205047607, -122.48387146)",37.7205,-122.484
3,Mixres,5087003,SCHLAGE LOCK,Yes,PL APPROVED,07/22/2014 12:00:00 AM,1679.0,1679.0,168.0,168.0,...,Visitacion Valley,MUG,Mixed Use,MUG,10,10 - South Bayshore,Schlage Lock,"(37.7116546631, -122.4022216797)",37.7117,-122.402
4,Mixres,9900048,PIER 48 / SEAWALL LOT 337,No,PL FILED,04/23/2013 12:00:00 AM,1500.0,1500.0,NaN,NaN,...,South of Market,M-2,Industrial,M-2,6,9 - South of Market,NaN,"(37.7757034302, -122.386138916)",37.7757,-122.386


In [129]:
len(dev)

126

In [123]:
dev['AFFORDABLE'].dropna(inplace=True)

In [127]:
dev['AFFORDABLE'].fillna(0, inplace=True)

In [128]:
for index, row in dev.iterrows():
    if row['AFFORDABLE']==0:
        dev.drop(index, axis=0, inplace=True)

In [133]:
# Create a geojson file export from the current dataframe
df_to_geojson(dev, filename='dev.geojson',properties=[
 'NAMEADDR',
 'BESTDATE',
 'UNITS',
 'UNITSNET',
 'AFFORDABLE',
 'AFFORDABLENET',
 'NEIGHBORHOOD',
 'ZONING_GEN',
 'PLAN_DISTRICT',
 'PLAN_AREA',], lat='lat', lon='lon')


{'feature_count': 126, 'filename': 'dev.geojson', 'type': 'file'}

In [96]:
dev['AFFORDABLE'].sort_values(ascending = False)

0       3345.0
1       2050.0
9        994.0
8        806.0
27       400.0
2        284.0
52       205.0
60       182.0
3        168.0
68       157.0
30       156.0
16       150.0
76       143.0
77       143.0
86       127.0
93       121.0
97       118.0
82       114.0
101      113.0
18       110.0
17       109.0
106      108.0
14       107.0
107      106.0
119       94.0
39        85.0
127       85.0
130       83.0
19        74.0
139       72.0
         ...  
1081       0.0
1082       0.0
1083       0.0
1084       0.0
1085       0.0
1064       0.0
1063       0.0
1062       0.0
1050       0.0
1041       0.0
1042       0.0
1043       0.0
1044       0.0
1045       0.0
1046       0.0
1047       0.0
1048       0.0
1049       0.0
1051       0.0
1061       0.0
1052       0.0
1053       0.0
1054       0.0
1055       0.0
1056       0.0
1057       0.0
1058       0.0
1059       0.0
1060       0.0
792        0.0
Name: AFFORDABLE, Length: 1585, dtype: float64

In [97]:
color_breaks = [10,100,1000,10000]
color_stops = create_color_stops(color_breaks, colors='YlOrRd')


In [145]:
color_breaks = [10, 100, 500,1000,10000]
color_stops = create_color_stops(color_breaks, colors='YlOrRd')

# Create the viz from the dataframe
viz = CircleViz('dev.geojson',
                access_token=token, 
                height='400px',
                color_property = 'AFFORDABLE',
                color_stops = color_stops,
                center = (-122.4, 37.75),
                zoom = 12,
               )



viz.show() 